# 爬取数据
参考github现成的代码，获取cookies后再调用接口爬取数据

原githu代码网址 https://github.com/longxiaofei/spider-BaiduIndex

In [2]:
import time
import pandas as pd
from qdata.baidu_index import get_search_index
from qdata.baidu_index.common import split_keywords


keywords_list = [['携程'], ['去哪儿'], ['同程网'],['马蜂窝'],['途牛网'],['穷游网'],['同城旅行'],['飞猪'],['艺龙']]
cookies = """BDUSS=jI0ZjFofkhWTGxqaVc1Zi16NnVHanpnTFV0Z3JBUmZIR05nb3d4ai1SVFJsRGxqSVFBQUFBJCQAAAAAAAAAAAEAAABn0QcceG5tYmMAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANEHEmPRBxJjQ;STOKEN=89bcba91b69b59ec6a2d5124eacf0153575c69543d258c6d08884024a6919bd7; """

data_df = pd.DataFrame()

for keywords in split_keywords(keywords_list):
    for index in get_search_index(
        keywords_list=keywords,
        start_date='2019-01-01',
        end_date='2022-08-01',
        cookies=cookies
    ):
        if index['type'] == 'all':
            data_df = pd.concat([data_df, data_df.from_dict(index)], ignore_index=True)
            # print(index)
    time.sleep(15)

data_df.to_csv('指数数据.csv',encoding='gbk',index=False)


# 存为月度数据
data_df['date']=pd.to_datetime(data_df.date) 
data_df['date']=data_df['date'].dt.to_period('M') 
data_df.groupby(['keyword','date']).mean().to_csv('月度数据.csv',encoding='gbk')

# 画图

timeline来源

https://matplotlib.org/stable/gallery/lines_bars_and_markers/timeline.html#sphx-glr-gallery-lines-bars-and-markers-timeline-py

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np
from matplotlib import rcParams
from datetime import datetime
# rcParams配置文件，用来定义各种变量的，这里用来定义字体
config = {
    "font.family":'serif',
    "font.size": 18,
    "mathtext.fontset":'stix', # 表示可以用latex写公式，stix类似于Time News Romen
    "font.serif": ['SimSun'], # 表示中文用的是宋体
    "figure.constrained_layout.use" : False,
}
rcParams.update(config) # 后更新配置，防止字体设置被覆盖
data_df=pd.read_csv('月度数据.csv',encoding='gbk')
data_df['date'] = [datetime.strptime(d, "%Y-%m") for d in data_df['date']]
# data_df['date'] = pd.to_datetime(data_df.date, format='%Y/%m/%d') 
# data_df['date'] = data_df['date'].dt.strftime('%Y-%m-%d')
fig, ax = plt.subplots(1,1)
# ax.xaxis.set_major_locator(ticker.MultipleLocator(5))

# plt.xticks(rotation=90)

# sns.lineplot(data=data_df, x='date', y='index', hue='keyword',style="keyword")
sns.lineplot(x='date',y='index',hue='keyword',data=data_df)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
# ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
plt.ylabel('log百度指数')
plt.show()